# TO GET STARTED: Enter a mission id below and then select Cell->Run All from the menu above

In [ ]:
mission_id = 25614

In [ ]:
import numpy as np
import os
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from urllib import parse

%matplotlib inline
FOLLOWER_URL = os.environ['FOLLOWER_URL']
                          
def make_connection():
    url_output = FOLLOWER_URL
    url = urlparse(url_output)
    conn = psycopg2.connect(
        database=url.path[1:],
        user=url.username,
        password=url.password,
        host=url.hostname
    )
    return conn

In [ ]:
def get_response(answer, transcription, q_type):
    if q_type in ['TextQuestion', 'TitleQuestion']:
        return str(answer[0])
    else:
        if transcription is not None:
            return str(transcription)
        else:
            return ""
cnc = make_connection()

snippets = pd.read_sql_query(
    """select id as snippet_id from snippets where mission_id = """
    + str(mission_id),
    cnc
)
snippet_list = (
    "(" + ",".join([str(a) for a in snippets["snippet_id"].tolist()]) + ")"
)
questions = pd.read_sql_query(
    """select id as question_id, position + 1 as question_num, type, part_id from questions where mission_id = """
    + str(mission_id) + """ and type in ('TitleQuestion','TextQuestion','VideoQuestion')"""
    ,
    cnc
)
question_list = (
    "(" + ",".join([str(a) for a in questions["question_id"].tolist()]) + ")"
)

part_list = (
    "(" + ",".join([str(a) for a in questions["part_id"].tolist()]) + ")"
)

parts = pd.read_sql_query(
    """select id as part_id, position + 1 as part_num from parts
    where id in """+ part_list,
    cnc
)
questions = pd.merge(questions, parts, on='part_id')
responses = pd.read_sql_query(
    """select id as response_id, question_id, answers from responses where snippet_id in """
    + snippet_list + """ and question_id in """ + question_list,
    cnc,
)

responses = pd.merge(responses, questions, on='question_id')
video_response_list = (
    "(" + ",".join([
        str(a) for a in responses[responses['type'] == 'VideoQuestion']
        ["response_id"].tolist()]) 
    + ")"
)  

transcripts = pd.read_sql_query(
    """select transcription, response_id from videos where
    response_id in """+ video_response_list,
    cnc
)
responses = pd.merge(
    responses,
    transcripts,
    on='response_id',
    how ='left'
)
responses['response'] = responses.apply(
    lambda row: get_response(row['answers'],row['transcription'], row['type']),
    axis=1
)
responses['word_count'] = responses.apply(
    lambda row: len(row['response'].split()),
    axis=1
)
cnc.close

# WORD COUNT STATS BY QUESTION

In [ ]:
responses.groupby(['part_num','question_num'])['word_count'].agg(['mean','median','std','sum','count']).reset_index()

# WOUNT COUNT STATS BY PART

In [ ]:
responses.groupby(['part_num'])['word_count'].agg(['mean','median','std','sum','count']).reset_index()

# WORD COUNT STATS BY QUESTION TYPE

In [ ]:
responses.groupby(['type'])['word_count'].agg(['mean','median','std','sum','count']).reset_index()

# WORD COUNT STATS BY MISSION

In [ ]:
responses['word_count'].agg(['mean','median','std','sum','count']).reset_index()